In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import re
import string

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
%cd /content/drive/MyDrive/Training/DTS_Tensorflow/demo/

/content/drive/MyDrive/Training/DTS_Tensorflow/demo


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gunzip cc.en.300.vec.gz
!rm -rf gunzip cc.en.300.vec.gz

--2022-07-26 05:48:47--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  34.2MB/s    in 70s     

2022-07-26 05:49:57 (18.1 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [ ]:
df = pd.read_csv('tweets.csv')

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+')
    return url.sub(r' httpsmark ', text)


def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)


def remove_atsymbol(text):
    name = re.compile(r'@\S+')
    return name.sub(r' atsymbol ', text)


def remove_hashtag(text):
    hashtag = re.compile(r'#')
    return hashtag.sub(r' hashtag ', text)


def remove_exclamation(text):
    exclamation = re.compile(r'!')
    return exclamation.sub(r' exclamation ', text)


def remove_question(text):
    question = re.compile(r'?')
    return question.sub(r' question ', text)


def remove_punc(text):
    return text.translate(str.maketrans('','',string.punctuation))


def remove_number(text):
    number = re.compile(r'\d+')
    return number.sub(r' number ', text)


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' emoji ', string)

In [ ]:

df['text'] = df['text'].str.lower()
df['text'] = df['text'].apply(lambda text: remove_URL(text))
df['text'] = df['text'].apply(lambda text: remove_html(text))
df['text'] = df['text'].apply(lambda text: remove_atsymbol(text))
df['text'] = df['text'].apply(lambda text: remove_hashtag(text))
df['text'] = df['text'].apply(lambda text: remove_exclamation(text))
df['text'] = df['text'].apply(lambda text: remove_punc(text))
df['text'] = df['text'].apply(lambda text: remove_number(text))
df['text'] = df['text'].apply(lambda text: remove_emoji(text))

In [ ]:
tk = Tokenizer(lower=True, filters='')
tk.fit_on_texts(df.text)

max_len = 120 # Calculate as max in dataset see 1.data_process.ipynb
train_tokenized = tk.texts_to_sequences(df.text)
X = pad_sequences(train_tokenized, maxlen=max_len)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,df.target, test_size=0.2, random_state=42)

In [ ]:
embed_size = 300
max_features = 30000

def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open("cc.en.300.vec"))
word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, embed_size))

for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
adam = Adam(learning_rate=0.001)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape = (max_len,)),
    tf.keras.layers.Embedding(nb_words+1, embed_size, weights = [embedding_matrix], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,recurrent_dropout=0.4)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(loss=SparseCategoricalCrossentropy(),optimizer=adam,metrics=[SparseCategoricalAccuracy()])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 300)          7230900   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              186880    
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 7,421,974
Trainable params: 191,074
Non-trainable params: 7,230,900
_________________________________________________________________


In [ ]:
callback = [EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    min_delta=0.01
), 
ModelCheckpoint(
    filepath='checkpoint_model_word2vec/',
    save_weights_only=True,
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True)
]

In [ ]:
# Fit model
history = model.fit(X_train,y_train,batch_size=512,epochs=20, validation_split=0.2,callbacks=callback)

Epoch 1/20
15/15 [==============================] - 47s 3s/step - loss: 0.5840 - sparse_categorical_accuracy: 0.7610 - val_loss: 0.4796 - val_sparse_categorical_accuracy: 0.8121
Epoch 2/20
15/15 [==============================] - 47s 3s/step - loss: 0.4554 - sparse_categorical_accuracy: 0.8109 - val_loss: 0.4266 - val_sparse_categorical_accuracy: 0.8115
Epoch 3/20
15/15 [==============================] - 45s 3s/step - loss: 0.4063 - sparse_categorical_accuracy: 0.8169 - val_loss: 0.3913 - val_sparse_categorical_accuracy: 0.8412
Epoch 4/20
15/15 [==============================] - 41s 3s/step - loss: 0.3656 - sparse_categorical_accuracy: 0.8492 - val_loss: 0.3566 - val_sparse_categorical_accuracy: 0.8549
Epoch 5/20
15/15 [==============================] - 37s 2s/step - loss: 0.3329 - sparse_categorical_accuracy: 0.8678 - val_loss: 0.3406 - val_sparse_categorical_accuracy: 0.8648
Epoch 6/20
15/15 [==============================] - 34s 2s/step - loss: 0.3140 - sparse_categorical_accuracy: 

In [ ]:
model.load_weights(f'checkpoint_model_word2vec/')

In [ ]:
model.evaluate(X_test, y_test, verbose=1)

72/72 [==============================] - 11s 156ms/step - loss: 0.2819 - sparse_categorical_accuracy: 0.8861


[0.2818973660469055, 0.8861038088798523]